In [6]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
import time
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
from fake_useragent import UserAgent  # For generating random user agents
import pandas as pd  # For data manipulation and creating DataFrames
import numpy as np
import concurrent.futures

In [8]:



keys = ['Corp','LLC','INC','LLP','LP']

active_id = "MainContent_rdbActive"
inactive_id = "MainContent_rdbInactive"


def get_all_active(current):  
    service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(service=service,options=options)
    driver.get('https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx')
    text_field = driver.find_element(By.ID, "MainContent_txtEntityName")
    text_field.clear()
    text_field.send_keys(current)

    search_type_dropdown = driver.find_element(By.ID, "MainContent_ddBeginsWithEntityName")
    select = Select(search_type_dropdown)
    select.select_by_visible_text("Full text")
    
    dropdown_element = driver.find_element(By.ID, "MainContent_ddRecordsPerPage")
    select1 = Select(dropdown_element)
    select1.select_by_visible_text("All items")

    
    time.sleep(10)
    middle_script = "window.scrollTo(0, document.body.scrollHeight / 2);"
    driver.execute_script(middle_script)
    
    # Wait to observe the scroll action
    time.sleep(10)
    
    # Scroll to the bottom of the page
    bottom_script = "window.scrollTo(0, document.body.scrollHeight);"
    driver.execute_script(bottom_script)
    
    #text_field.send_keys(Keys.RETURN)
    
    time.sleep(10)
    
    #text_field.send_keys(Keys.RETURN)
    
    time.sleep(5)
    page = driver.page_source
    
    soup = BeautifulSoup(page, "html.parser")
    soup = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
    header = []
    for i in soup.find_all('th'):
        title = i.text
        header.append(title)
    companies = pd.DataFrame(columns = header)
    for j in soup.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(companies)
        companies.loc[length] = row
    companies['status'] = 'active'
    companies['entity_name'] = current
    companies.to_csv(f"companies_active_{current}.csv")
    print(f'All Actives for {current} ssuccessfully scraped')
    driver.quit()


def get_all_inactive(current):  
    service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    
    driver = webdriver.Chrome(service=service,options=options)
    driver.get('https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx')

    button = driver.find_element(By.ID, inactive_id)


    button.click()
    text_field = driver.find_element(By.ID, "MainContent_txtEntityName")
    text_field.clear()
    text_field.send_keys(current)
    
    search_type_dropdown = driver.find_element(By.ID, "MainContent_ddBeginsWithEntityName")
    select = Select(search_type_dropdown)
    select.select_by_visible_text("Full text")
    
    dropdown_element = driver.find_element(By.ID, "MainContent_ddRecordsPerPage")
    select1 = Select(dropdown_element)
    select1.select_by_visible_text("All items")


    time.sleep(10)
    middle_script = "window.scrollTo(0, document.body.scrollHeight / 2);"
    driver.execute_script(middle_script)
    
    # Wait to observe the scroll action
    time.sleep(10)
    
    # Scroll to the bottom of the page
    bottom_script = "window.scrollTo(0, document.body.scrollHeight);"
    driver.execute_script(bottom_script)
    
    #text_field.send_keys(Keys.RETURN)
    
    time.sleep(10)
    page = driver.page_source
    
    soup = BeautifulSoup(page, "html.parser")
    soup = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
    header = []
    for i in soup.find_all('th'):
        title = i.text
        header.append(title)
    companies = pd.DataFrame(columns = header)
    for j in soup.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(companies)
        companies.loc[length] = row
    companies['status'] = 'inactive'
    companies['entity_name'] = current
    companies.to_csv(f"companies_inactive_{current}.csv")
    print(f'All Inactives for {current} successfully scraped')
    driver.quit()
    companies

for i in keys:
    get_all_active(i)
    
for i in keys:
    get_all_inactive(i)



KeyboardInterrupt: 